In [11]:
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
data = pd.read_csv('the-reddit-climate-change-dataset-comments.csv')
data = data.dropna() # drop any rows with missing values

In [17]:
# print(data.iloc[0:20])
print(data.size)

45435670
